## A Guide to TF Layers: Building a Convolutional Neural Network 

The TensorFlow layers module provides a high-level API that makes it easy to construct a neural network. It provides methods that facilitate the creation of dense (fully connected) layers and convolutional layers, adding activation functions, and applying dropout regularization. In this tutorial, you'll learn how to use layers to build a convolutional neural network model to recognize the handwritten digits in the MNIST data set.

The MNIST dataset comprises 60,000 training examples and 10,000 test examples of the handwritten digits 0–9, formatted as 28x28-pixel monochrome images.

### Getting Started

Let's set up the skeleton for our TensorFlow program.

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [ ]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # input layer
    """for creating convolutional and pooling layers for two-dimensional image data expect input tensors to have a shape of [batch_size, image_width, image_height, channels]"""
    """indicated -1 for batch size, which specifies that this dimension should be dynamically computed based on the number of input values in features["x"]"""
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])    
    
    #convolutional layer #1
    # dimensions of the filters as [width, height] (here, [5, 5])
    conv1 = tf.layers.conv2d(inputs=input_layer, filters=32, kernel_size=[5, 5],
                             padding="same", activation=tf.nn.relu)
    """output tensor produced by conv2d() has a shape of [batch_size, 28, 28, 32]: the same width and height dimensions as the input, but now with 32 channels holding the output from each of the filters"""
    
    #pooling layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2],
                                    strides=2)
    
    #convolutional layer #2 and pooling layer #2
    conv2 = tf.layers.conv2d(inputs=pool1, filters=64, 
                             kernel_size=[5, 5], padding="same", activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2],
                                    strides=2)
    
    #dense layer
    """Each example has 7 (pool2 width) * 7 (pool2 height) * 64 (pool2 channels) features, so we want the features dimension to have a value of 7 * 7 * 64 (3136 in total). The output tensor, pool2_flat, has shape [batch_size, 3136]"""
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.4,
                                training=mode == tf.estimator.ModeKeys.TRAIN)

    #logits layer
    logits = tf.layers.dense(inputs=dropout, units=10)    
    #logits layer of our model returns our predictions as raw values in a [batch_size, 10]-dimensional tensor.
    
    #generate predictions
    predictions = {
        #predicted class 
        "classes": tf.argmax(input=logits, axis=1),
        #derive probabilities from our logits layer by applying softmax activation
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    #compile our predictions in a dict, and return an EstimatorSpec object
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    #calculate Loss (for both TRAIN and EVAL modes)
#     loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)    
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    """takes onehot_labels and logits as arguments, performs softmax activation on logits, calculates cross-entropy, and returns our loss as a scalar Tensor"""    
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)

    #configure the Training Op (for TRAIN mode)
    #configure our model to optimize this loss value during training
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(
                                    labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss,
                                      eval_metric_ops=eval_metric_ops)


### Training and Evaluating the CNN MNIST Classifier

First, let's load our training and test data. Add a main() function

In [ ]:
def main(unused_argv):
    #load training and eval data
    mnist = tf.contrib.learn.datasets.load_dataset("mnist")
    train_data = mnist.train.images #returns np.array
    train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
    eval_data = mnist.test.images #returns np.array
    eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    
    #create estimator(a TensorFlow class for performing high-level model training, evaluation, and inference)
    mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, 
                                             model_dir="/tmp/mnist_convnet_model")
    #set up logging for predictions so we can track progress during training 
    #log the values in the "Softmax" tensor with label "probabilities"
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

    #train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": train_data},
                        y=train_labels, batch_size=100, num_epochs=None, shuffle=True)
    mnist_classifier.train(input_fn=train_input_fn, steps=20000, hooks=[logging_hook]) 
    
    #evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": eval_data},
                       y=eval_labels, num_epochs=1, shuffle=False)
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)    

In [ ]:
if __name__ == "__main__":
    tf.app.run()